In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 2.8 MB/s 
     |████████████████████████████████| 3.3 MB 44.9 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Load the data
path = '/content/drive/MyDrive/GPT-2/'
with open(path + 'train.txt') as f:
  train_data = f.read()
with open(path + 'test.txt') as f:
  test_data = f.read()

In [ ]:
print(f'Train data length: {len(train_data)}')
print(f'Test data length: {len(test_data)}')

Train data length: 113046
Test data length: 28330


In [ ]:

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = path + 'train.txt'
test_path = path + 'test.txt'
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer);

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


**Initialize Trainer with TrainingArguments and GPT-2 model**

In [ ]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2_output", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    save_steps=500, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

**Train and save the model**

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 189
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 36


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=36, training_loss=3.934901343451606, metrics={'train_runtime': 40.6501, 'train_samples_per_second': 13.948, 'train_steps_per_second': 0.886, 'total_flos': 37038145536000.0, 'train_loss': 3.934901343451606, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to ./gpt2_output
Configuration saved in ./gpt2_output/config.json
Model weights saved in ./gpt2_output/pytorch_model.bin


**Test the model**

In [ ]:
from transformers import pipeline

gpt = pipeline('text-generation',model='./gpt2_output', tokenizer='gpt2')

loading configuration file ./gpt2_output/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "use_cache": true,
  "vocab_size": 50257
}

loading configuration file ./gpt2_output/config.json
Model config GPT2Config {
  "_name_or_path

In [ ]:
gpt("To ensure a smooth transition of the employee from the company.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'To ensure a smooth transition of the employee from the company.\n\nCrowdfunding\n\nAfter a short period of time the fundraising will begin through the campaign. It is not possible to fundraise through the website but donations can be sent by email'}]